# Реализация игры Grid World с помощью Q-learning

--------------------------------------------------

## Правила игры
![Img](https://raw.githubusercontent.com/DPritykin/Control-Theory-Course/main/Term%202%20Nonlinear%20Dynamics%20and%20Control/Images/gridworld.png)

<font size="3">  Агент начинает игру в произвольном свободном положении(на примере слева снизу поля) и заканчивает в ячейке с  +1 ("сектор приз") либо -1 ("сектор штраф"). 
    На каждом шагу он может двигаться вертикально или горизонтально, но не может проходить через препятствия или уходить за пределы игрового поля.</font>

</font> <font size="3">Реализация выбранного действия на каждом шаге не детерминирована, то есть агент, принимая решение перейти на соседнюю клеточку (например, наверх), оказывается на ней с вероятностью 0.8, оставшаяся вероятность равномерно распределяется между перпендикулярными направлениями движения (то есть, если агент принял решение пойти наверх, то перпендикулярные направления - вправо-влево).</font>

--------------------------------------------------

## Поле
<font size="3">Зададим переменные, определяющие поле для игры</font>

In [1]:
import numpy as np

BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3)
LOSE_STATE = (1, 3)
START = (2, 0)
DETERMINISTIC = False

<font size="3">Класс State определяет параметры текущего состояния, а также возвращает следущее состояние в зависимости от выбранного действия</font>

In [2]:
class State:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1, 1] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC

    # Определяет награду согласно текущему состоянию    
    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0
    
    # Определяет, является ли текущее состояние state терминальным
    def isEndFunc(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True

    # выбор реализуемого действия после принятия решения (action) с учётом недетерминированности модели        
    def _chooseActionProb(self, action):
        if action == "up":
            return np.random.choice(["up", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "down":
            return np.random.choice(["down", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "left":
            return np.random.choice(["left", "up", "down"], p=[0.8, 0.1, 0.1])
        if action == "right":
            return np.random.choice(["right", "up", "down"], p=[0.8, 0.1, 0.1])

    # переход в следующее состояние после выбора действия (action)    
    def nxtPosition(self, action):
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position on board
        """
        if not self.determine:
            action = self._chooseActionProb(action)
            
        if action == "up":
            nxtState = (self.state[0] - 1, self.state[1])
        elif action == "down":
            nxtState = (self.state[0] + 1, self.state[1])
        elif action == "left":
            nxtState = (self.state[0], self.state[1] - 1)
        else:
            nxtState = (self.state[0], self.state[1] + 1)

        # Проверяет, лежит ли состояние state в границах игрового поля
        if (nxtState[0] >= 0) and (nxtState[0] <= 2):
            if (nxtState[1] >= 0) and (nxtState[1] <= 3):
                if nxtState != (1, 1):
                    return nxtState

        return self.state

    # вывод игрового поля
    def showBoard(self):
        self.board[self.state] = 1
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-----------------')

## Алгортим обучения агента

<font size="3">1. Рассмотрим кортеж $(s, a, r, s')$:</font>

<font size="3">2. Вычислим промежуточное значение функции качества:</font>

### $\hat{Q}(s, a) = R(s,a,s') + \gamma \cdot \max_{a}{Q_k(s',a')}$

<font size="3">3. Будем вычислять новую оценку для Q-функции как взвешенную сумму имеющейся оценки и новой информации $\hat{Q}(s,a)$:</font>

### $Q_{k+1}(s, a) = (1-\alpha) \cdot Q_{k}(s, a) + \alpha \cdot \hat{Q}(s,a) =  Q_{k}(s, a) + \alpha \cdot (\hat{Q}(s,a) - Q_{k}(s, a))$

In [3]:
class Agent:

    def __init__(self):
        self.states = []        # инициализация 
        self.actions = ["up", "down", "left", "right"]
        self.State = State()
        self.isEnd = self.State.isEnd
        self.lr = 0.2            # learning rate - скорость обучения \alpha
        self.exp_rate = 0.3      # exploration rate - вероятность с которой происходит исследование (exploration vs eploitation)
        self.decay_gamma = 0.9   # показатель степени для вычисления кумулятивного вознаграждения

        # initial Q values
        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0  # Q value is a dict of dict

    # выбор действия в текущем состоянии
    def chooseAction(self):
        # choose action with most expected value
        mx_nxt_reward = 0
        action = ""

        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # жадный выбор
            for a in self.actions:
                current_position = self.State.state
                nxt_reward = self.Q_values[current_position][a]
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward
            # print("current pos: {}, greedy aciton: {}".format(self.State.state, action))
        return action

    # выполнить действие и перейти в следующее состояние
    def takeAction(self, action):
        position = self.State.nxtPosition(action)
        # update State
        return State(state=position)

    # сброс истории
    def reset(self):
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd

    # играть определённое количество раундов и обновлять Q на обратном проходе вдоль каждой траектории
    def play(self, rounds=10):
        i = 0
        while i < rounds:
            # to the end of game back propagate reward
            if self.State.isEnd:
                # back propagate
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                print("Game End Reward", reward)
                for s in reversed(self.states):
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr * (self.decay_gamma * reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()
                # append trace
                self.states.append([(self.State.state), action])
                print("current position {} action {}".format(self.State.state, action))
                # by taking the action, it reaches the next state
                self.State = self.takeAction(action)
                # mark is end
                self.State.isEndFunc()
                print("nxt state", self.State.state)
                print("---------------------")
                self.isEnd = self.State.isEnd

In [4]:
if __name__ == "__main__":
    ag = Agent()
    print("initial Q-values ... \n")
    print(ag.Q_values)

    ag.play(500)
    print("latest Q-values ... \n")
    print(ag.Q_values)

initial Q-values ... 

{(0, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (0, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (0, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (0, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (1, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (1, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (1, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (1, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (2, 0): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (2, 1): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (2, 2): {'up': 0, 'down': 0, 'left': 0, 'right': 0}, (2, 3): {'up': 0, 'down': 0, 'left': 0, 'right': 0}}
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action down
nx

---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
---------------------
current position (2, 0) action down
nxt state (2, 0)
---------------------
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action up
nxt state (2, 1)
---------------------
current position (2, 1) action left
nxt state (2, 1)
---------------------
current position (2, 1) action right
nxt state (2, 2)
---------------------
current position (2, 2) action up
nxt state (1, 2)
---------------------
curre

KeyError: ''